# Forecasting

In [ ]:
#load hourly resampled & merged df
#import pandas as pd
import urllib.request
urllib.request.urlretrieve('https://docs.google.com/uc?export=download&id=1--eG9u-siOXAwVvB_P_t4gxkzYcwILMa', 'merge_export41_meteoLC102_hourly.csv')
#df_hourly = pd.read_csv('merge_export41_meteoLC102_hourly.csv', index_col = 0)
#df_hourly.head()

## Distinguishing between true missing values and no-event 'observations'
The time series seems to contain many missing values for noise event variables. Not all of these are true missing values, since not registering an event may simply mean there is no event. In an effort to distinguish between true missing values and no-event 'observations', we first determine the missing time periods. Considering only the gaps > 1 day, we note the following:

for most locations, gaps are nearly always >= 6 days, the few exceptions have a length of 1-2 days. These exceptions may still (largely) consist of true no-event 'observations'

-> (arbitrary) cutoff at 2 days?

MP08bis - Vrijthof is the anomaly with 55 missing periods (vs max 5 for other locations), half of which are < 3 days, 75% of which are < 6 days. This is likely the result of its location vs that of the other locations (courtyard of the Town Hall vs along the Naamsestraat).

-> (very arbitrary) cutoff at 7 days?

On another note: Interestingly, most of the (supported) events registered at Vrijthof are classified as human singing. This may be linked to the nearby presence of Het Radiohuis.

In [ ]:
import numpy as np
from datetime import timedelta

def add_end(ts):
  upper = pd.DataFrame(ts.iloc[0:1,:].replace([0,1], np.nan))   
  upper['begin_date'] = pd.to_datetime('2023-01-01 00:00:00')
  return pd.concat([ts, upper])

def missing_or_noevent(ts, cutoff = 2, cutoff_Vrijthof = 7):
  '''
  This function fills some of the nans in the noise event variables with zeroes, based on the specified cutoff values. 
  The argument 'cutoff_Vrijthof' is used to specify the maximum length of a time period with missing values (in days!)
  before it is considered truly missing for MP08bis, 'cutoff' does the same for all other MPs.
  It returns two DataFrames: the adapted input DataFrame, and a DataFrame with all the missing time periods. 
  The latter is not filtered by the specified cutoffs. 
  'Unsupported' category is assumed to consist of unclassifiable events and is thus treated as an event, not missing.
  '''

  # Get time series sampling frequency
  ts['timestamp'] = pd.to_datetime(ts['timestamp'])
  ts = ts.sort_values(['location_csv','timestamp']).reset_index(drop=True)
  freq = pd.to_timedelta(ts.loc[1, 'timestamp'] - ts.loc[0, 'timestamp'])

  # Construct a df with the missing time periods
  missing_time = pd.DataFrame()
  df = ts.dropna(subset = 'human_noise')
  missing_time[['location_csv', 'begin_date']] = df[['location_csv', 'timestamp']]
  missing_time = missing_time.groupby('location_csv').apply(add_end).reset_index(drop=True)
  missing_time = missing_time.sort_values(["location_csv","begin_date"]).reset_index(drop=True)
  missing_time['end_date'] = missing_time['begin_date'].shift(-1)
  missing_time['begin_date'] = missing_time['begin_date'] + freq
  missing_time['timedelta'] = missing_time.groupby("location_csv")["begin_date"].diff().shift(-1)
  missing_time = missing_time.dropna()
  missing_time = missing_time[missing_time['timedelta'] > freq]
  missing_time['timedelta'] = missing_time['timedelta'] - freq

  # Filter by cutoff values
  true_na_vh = missing_time.loc[(missing_time['timedelta'] > timedelta(days = cutoff_Vrijthof)) & (missing_time['location_csv'] == '280324_mp08bis---vrijthof.csv')]
  true_na_other = missing_time.loc[(missing_time['timedelta'] > timedelta(days = cutoff)) & (missing_time['location_csv'] != '280324_mp08bis---vrijthof.csv')]


  col_to_fill = ts.columns[ts.columns.str.contains('noise')].values.tolist()

  ts_vh = ts.loc[ts['location_csv'] == '280324_mp08bis---vrijthof.csv' ].copy()
  ts_other = ts.loc[ts['location_csv'] != '280324_mp08bis---vrijthof.csv'].copy()

  
  # Add column true_na yes/no (less intervals to check for true nans than false nans)
  # If timestamp not in true_na, replace any nans with 0s

    #Vrijthof
  ts_vh['true_na'] = ts_vh['timestamp'].apply(lambda t: any((true_na_vh["begin_date"] <= t) & (true_na_vh["end_date"] > t)))
  ts_vh.loc[ts_vh['true_na'] == 0, col_to_fill] = ts_vh.loc[ts_vh['true_na'] == 0, col_to_fill].fillna(0)

    #other MPs
  other = ts_other['location_csv'].drop_duplicates().tolist()
  for MP in other:
    ts_other.loc[ts_other['location_csv'] == MP,'true_na'] = ts_other.loc[ts_other['location_csv'] == MP, 'timestamp'] \
                          .apply(lambda t: any((true_na_other['location_csv'] == MP) &(true_na_other["begin_date"] <= t) & (true_na_other["end_date"] > t)))
  ts_other.loc[ts_other['true_na'] == 0, col_to_fill] = ts_other.loc[ts_other['true_na'] == 0, col_to_fill].fillna(0)

  df = pd.concat([ts_vh, ts_other]).drop('true_na', axis = 1)  

  return df, missing_time

df, missing = missing_or_noevent(df_hourly)

## Feature building
Could still be added: window features, such as 'number of time periods with noise event, out of the last 6 time periods' 

In [ ]:
import holidays
from datetime import timedelta

# DEV: 
## GPT prompt:
## if I have a pandas dataframe with timestamps (UTC), can you suggest some additional features to engineer? (for example: weekend, weekday, Belgian holiday, season, ...)
## Can you provide python3 code on an imaginary dataframe?

df["timestamp"] = pd.to_datetime(df["timestamp"])

# Add weekday/weekend feature
df["is_weekend"] = df["timestamp"].dt.weekday.isin([5, 6]).astype(int)

# Add day of the week feature
df["day_of_week"] = df["timestamp"].dt.day_name()

# Add hour of the day feature
df["hour_of_day"] = df["timestamp"].dt.hour

# Add time of day feature
df["time_of_day"] = pd.cut(
    df["timestamp"].dt.hour,
    bins=[-1, 6, 12, 18, 24],
    labels=["Night", "Morning", "Afternoon", "Evening"],
)  # equal bins

# Add season feature
df["season"] = pd.cut(
    df["timestamp"].dt.month,
    bins=[0, 3, 6, 9, 12],
    labels=["Winter", "Spring", "Summer", "Fall"],
)

# Add month feature
df["month"] = df["timestamp"].dt.month_name()

# Add day of the month feature
df["day_of_month"] = df["timestamp"].dt.day

# Add quarter feature
df["quarter"] = "Q" + df["timestamp"].dt.quarter.astype(str)

# Add Belgian holidays feature
be_holidays = holidays.BE()
df["is_be_holiday"] = (
    df["timestamp"].dt.date.astype("datetime64").isin(be_holidays).astype(int)
)

# Add business day feature
df["is_business_day"] = ~df["timestamp"].dt.weekday.isin([5, 6]) & ~df["is_be_holiday"]


In [ ]:
# Add feature indicating whether the response variable is missing
df['missing'] = df['human_noise'].isna()

# Create dataframe with exam & vacation dates based on the academic calendars 2021-2022 & 2022-2023 of the KU Leuven & KU Leuven Group T 
# https://www.kuleuven.be/over-kuleuven/kalenders/kalenders-21-22 & https://www.kuleuven.be/over-kuleuven/kalenders 
# vacation = periods with no lessons or exams lasting at least 1 week
kul_ac_year = pd.DataFrame({"begin_date": ["2022-01-10", "2022-05-30", "2022-01-10", "2022-05-30",
                                           "2022-01-31", "2022-06-27", "2022-01-01", "2022-02-05",
                                           "2022-04-02", "2022-07-02", "2022-12-24"],
                    "end_date": ["2022-02-04", "2022-07-01", "2022-01-30", "2022-06-26", 
                                 "2022-02-04", "2022-07-01", "2022-01-13", "2022-02-13", 
                                 "2022-04-18", "2022-09-25", "2022-12-31"],
                    "type": ["exams", "exams", "first_exam_weeks", "first_exam_weeks", 
                             "final_exam_week", "final_exam_week", "vacation", "vacation", 
                             "vacation", "vacation", "vacation"]})

kul_ac_year["begin_date"] = pd.to_datetime(kul_ac_year["begin_date"])
kul_ac_year["end_date"] = pd.to_datetime(kul_ac_year["end_date"])
kul_ac_year["end_date"] = kul_ac_year["end_date"] + pd.Timedelta('23:59:59')

kul_ac_year

# Add (university) exams feature (only first & second exam period)
df["exams"] = df["timestamp"].apply(lambda t: any((kul_ac_year["type"] == "exams") & (kul_ac_year["begin_date"] <= t) & (kul_ac_year["end_date"] >= t)))

  # Alternatively add 2 features, one for first, 'normal', exam weeks, one for the last week
df["first_exam_weeks"] = df["timestamp"].apply(lambda t: any((kul_ac_year["type"] == "first_exam_weeks") & (kul_ac_year["begin_date"] <= t) & (kul_ac_year["end_date"] >= t)))
df["final_exam_week"] = df["timestamp"].apply(lambda t: any((kul_ac_year["type"] == "final_exam_week") & (kul_ac_year["begin_date"] <= t) & (kul_ac_year["end_date"] >= t)))

# Add student vacation periods feature
df["student_vacation"] = df["timestamp"].apply(lambda t: any((kul_ac_year["type"] == "vacation") & (kul_ac_year["begin_date"] <= t) & (kul_ac_year["end_date"] >= t)))
df.head()

## Forecasting with skforecast and LightGBM
Code used as blueprint:
Forecasting time series with gradient boosting: Skforecast, XGBoost, LightGBM y CatBoost by Joaquín Amat Rodrigo and Javier Escobar Ortiz, available under a Attribution 4.0 International (CC BY 4.0) at https://www.cienciadedatos.net/documentos/py39-forecasting-time-series-with-skforecast-xgboost-lightgbm-catboost.html

In [ ]:
import pandas as pd
import numpy as np

from lightgbm import LGBMClassifier

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.metrics import recall_score, precision_score, f1_score

from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries
from skforecast.utils import save_forecaster, load_forecaster

In [ ]:
## Create input dfs
# Drop location 08bis due to no human noise events occuring during validation month (November)
df = df[df['location_csv'] != '280324_mp08bis---vrijthof.csv']

# Create separate df with target time series 
df['timestamp'] = pd.to_datetime(df['timestamp'])
ts_data = pd.pivot_table(data = df, values = 'human_noise', index = 'timestamp', columns = 'location_csv')

# Additional category for missing response -> multiclass for some locations
ts_data = ts_data.fillna(2)
ts_data = ts_data.astype('category')

# Drop last row (not real data) 
ts_data.drop(index=ts_data.index[-1],axis=0,inplace=True)

# Explicitly set hourly frequency of datetime index
ts_data.index.freq = 'H'

# Create dictionary with exogenous data for every location 
# necessary because missing values are location-specific
locations = df['location_csv'].drop_duplicates().tolist()
exog_datasets = {}

for location in locations:
  # Subset by location
  somename = df.loc[df['location_csv'] == location,:]

  # Create df with exogenous vars
  cat_exogs = ['missing',
              'is_weekend',
              'day_of_week',
              'time_of_day',
              'is_be_holiday',
              'is_business_day',
              'season',
              'month',
              'first_exam_weeks', 
              'final_exam_week',
              'student_vacation']
  num_exogs = ['hour_of_day',
              'LC_TEMP_QCL3', 
              'LC_WINDSPEED', 
              'LC_RAININ']

  exog_data  = somename[cat_exogs + num_exogs + ['timestamp']].set_index('timestamp').sort_index()
  exog_data[cat_exogs] = exog_data[cat_exogs].astype('category')

  # Drop last row (not real data) 
  exog_data.drop(index=exog_data.index[-1],axis=0,inplace=True)

  # Explicitly set hourly frequency of datetime index
  exog_data.index.freq = 'H'

  # Add to dictionary
  exog_datasets[location] = exog_data

### Gridsearch

In [ ]:
## Dependent multivariate time series analysis with direct multi-step forecasting

# Defining train-val-test-split dates,
# starting from the time the last sensor began registering events of any type
# to minimize missing values in response var and lag predictors
start_train = '2022-03-07 22:00:00' 
end_train = '2022-10-31 23:59:00'
end_val = '2022-11-30 23:59:00'

# Different class imbalance per location -> different weights
# -> create custom weights function for every location 
# (skforecast only allows a function based on datetime index, not simply a column)
def createFunc(location):
  ts_train = ts_data.loc[start_train:end_train, location]
  ts = ts_data.loc[start_train:, location]
  minority_weight = ts_train.value_counts()[0] / ts_train.value_counts()[1]
  weights = np.where(ts == 1, minority_weight, 1)

  def custom_weights(index):
    w = weights[:len(index)]
    return w

  return custom_weights

# Create dictionary of weight functions
weight_funcs = [createFunc(location) for location in locations]
weight_funcs = {k:v for k,v in zip(locations, weight_funcs)}

# Define metric to calculate f1-score for positive label regardless of whether there's an additional 'missing' class
def f1score_pos(y_true, y_pred):
  scores = f1_score(y_true, y_pred, average = None)
  return scores[1]

# Transform categorical features using ordinal encoding. 
# Numeric features are left untouched. 
# if a new category is found in the test set, it is encoded as -1.

pipeline_cat = make_pipeline(OrdinalEncoder(dtype=int, #int dtype required by skforecast 
                                            handle_unknown="use_encoded_value",
                                            unknown_value=-1,),
                             FunctionTransformer(func=lambda x: x.astype('category'), 
                                                 feature_names_out = 'one-to-one'))
transformer_exog = make_column_transformer((pipeline_cat, cat_exogs),
                                           remainder="passthrough").set_output(transform="pandas")

# Define max lags
lags = 48

# Define forecast horizon
h = 12

# Define grids
lags_grid = {12, 24, 36, 48}
param_grid = {'min_child_samples': [10, 20, 30],
              'num_leaves': [5, 10, 15, 20, 25]}

# Gridsearch for all locations
LGBM_gs = {}
for level in locations:
  
  # Initializing the forecaster
  forecaster = ForecasterAutoregMultiVariate(regressor = LGBMClassifier(categorical_features='auto', random_state=123, max_depth = 6),
                                            lags = lags,
                                            level = level,
                                            steps = h, 
                                            transformer_exog = transformer_exog,
                                            weight_func = weight_funcs[level]
  )

  # Grid search
  result = grid_search_forecaster_multiseries(forecaster = forecaster,
                                                series = ts_data.loc[start_train:end_val, :],
                                                exog = exog_datasets[level].loc[start_train:end_val],
                                                lags_grid = lags_grid,
                                                param_grid = param_grid,
                                                steps = h,
                                                metric = f1score_pos,
                                                initial_train_size = len(ts_data.loc[start_train:end_train]),
                                                refit = False,
                                                fixed_train_size = False,
                                                return_best = False,
                                                verbose = False
  )      
  #print(result.to_string())
  LGBM_gs[level] = result

In [ ]:
for level in locations:
  gs = LGBM_gs[level]
  best = gs.sort_values('f1score_pos', ascending = False).reset_index().loc[0, 'f1score_pos']
  print(level, best)

### Testing the model & fitting it on all data

In [ ]:
# Defining train-val-test-split dates,
# starting from the time the last sensor began registering events of any type
# to minimize missing values in response var and lag predictors
start_train = '2022-03-07 22:00:00' 
end_train = '2022-10-31 23:59:00'
end_val = '2022-11-30 23:59:00'

# Define metric to calculate f1-score for every label separately
def f1score(y_true, y_pred):
  scores = f1_score(y_true, y_pred, average = None)
  return scores

# Define function to change column type to categorical, since lambda function cannot be saved by built-in method
def to_cat(x):
  x = x.astype('category')
  return x

# Transform categorical features using ordinal encoding. 
# Numeric features are left untouched. 
# if a new category is found in the test set, it is encoded as -1.
pipeline_cat = make_pipeline(OrdinalEncoder(dtype=int, # int dtype required by skforecast 
                                            handle_unknown="use_encoded_value",
                                           unknown_value=-1,),
                             FunctionTransformer(func = to_cat, 
                                                 feature_names_out = 'one-to-one'))
transformer_exog = make_column_transformer((pipeline_cat, cat_exogs),
                                           remainder="passthrough").set_output(transform="pandas")

# Define forecast horizon
h = 12 

# Testing best model for each location 
# And fitting each of those on all data
metrics_best = pd.DataFrame()
predictions_best = {}
forecasters = {}

for level in locations:
  gs = LGBM_gs[level]
  min_child_samples = gs.sort_values('f1score_pos', ascending = False).reset_index().loc[0, 'min_child_samples']
  num_leaves = gs.sort_values('f1score_pos', ascending = False).reset_index().loc[0, 'num_leaves']
  lags = gs.sort_values('f1score_pos', ascending = False).reset_index().loc[0, 'lags']

  # Custom_weights function moved inside loop since nested function cannot be saved
  def get_custom_weights(index):
    ts_train = ts_data.loc[start_train:end_val, level]
    ts = ts_data.loc[start_train:, level]
    minority_weight = ts_train.value_counts()[0] / ts_train.value_counts()[1]
    weights = np.where(ts == 1, minority_weight, 1)
    w = weights[:len(index)]
    return w

  # Initializing the forecaster
  forecaster = ForecasterAutoregMultiVariate(regressor = LGBMClassifier(categorical_features='auto', 
                                                                        random_state=123,
                                                                        min_child_samples = min_child_samples,
                                                                        num_leaves = num_leaves,
                                                                        max_depth = 6
                                                                        ),
                                            lags = lags,
                                            level = level,
                                            steps = h, 
                                            transformer_exog = transformer_exog,
                                            weight_func = get_custom_weights
  )
  
  # Testing best model for each location 
  metric_best, preds_best = backtesting_forecaster_multiseries(forecaster = forecaster,
                                                series = ts_data.loc[start_train:],
                                                exog = exog_datasets[level].loc[start_train:],
                                                steps = h,
                                                metric = f1score,
                                                initial_train_size = len(ts_data.loc[start_train:end_val]),
                                                refit = False,
                                                fixed_train_size = False,
                                                verbose = False
  )
  metrics_best = pd.concat([metrics_best, metric_best])
  predictions_best[level] = preds_best 

  # Training best model for each location on all data
  forecaster.fit(series = ts_data.loc[start_train:],
                 exog = exog_datasets[level].loc[start_train:])    

  forecasters[level] = forecaster 
  name = level[:-4]
  save_forecaster(forecaster, file_name = f'/forecasters/forecaster_h12_{name}.pkl')
 

In [ ]:
metrics_best
# many f1-scores for positive label are 0 due to either no true postives, 
# or no predicted positives (undefined f1-score defaults to 0)
# If missing values are a label, its f1-score = 1, 
# indicating that adding the missing-value variable worked as intended

### How to load forecasters & use them to make predictions

In [ ]:
# Load all forecasters
loaded_forecasters = {}
for level in locations:
  name = level[:-4]
  loaded_forecaster = load_forecaster(file_name = f'/forecasters/forecaster_h12_{name}.pkl', verbose = False)
  loaded_forecasters[level] = loaded_forecaster

# Create dataframe with all of the exogeneous variables that were used to train model
# values for weather can/should be made adaptable through sliders 
# 99% sure that you can use the same exog dataframe for all locations
# here I just take a part of a training exog_data and overwrite its timestamps, obviously don't do that in the final implementation
exog = exog_datasets['255443_mp-06-parkstraat-2-la-filosovia.csv'][500:512].reset_index().set_index(pd.date_range(start = '2023.01.01 00:00:00', periods = 12, freq = 'H'))

# Generate predictions for the next 12 hours for one location
loaded_forecasters['255439_mp-01-naamsestraat-35-maxim.csv'].predict(steps = 12, exog = exog)

# Generate prediction for the 7th hour
loaded_forecasters['255439_mp-01-naamsestraat-35-maxim.csv'].predict(steps = [7], exog = exog)